In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
print(zipcodes.crs)
zipcodes.head( )

In [ ]:
type(zipcodes)

In [ ]:
zipcodes.plot();

In [ ]:
#geopandas will take care of legend styling if you pass a dict of keywords

leg_kwds = {'title': 'Zipcodes', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 2}

zipcodes.plot(column = 'zip', figsize=(10, 10),
              edgecolor = 'black',
              legend = True, legend_kwds = leg_kwds, 
              cmap = 'Set2')
plt.show()

###  Metro Nashville Arts Commission or “Metro Arts” is the office of Arts & Culture for the city of Nashville and Davidson County. We believe that arts drive a more vibrant and equitable community. We strive to ensure that all Nashvillians have access to a creative life through community investments, artist and organizational training, public art and creative placemaking coordination and direct programs that involve residents in all forms of arts and culture.

Metro Nashville Arts Commission was started in 1978 by Metro Charter.

In [ ]:
 Metro_Art_in_Public_Places = pd.read_csv('../data/Metro_Art_in_Public_Places.csv')
print( Metro_Art_in_Public_Places.shape)
 Metro_Art_in_Public_Places.head(3)

In [ ]:
 Metro_Art_in_Public_Places.info()

In [ ]:
 Metro_Art_in_Public_Places.columns

In [ ]:
zipcodes.crs

In [ ]:
 Metro_Art_in_Public_Places_geo = gpd.GeoDataFrame( Metro_Art_in_Public_Places, 
                           crs = zipcodes.crs, 
                           geometry = gpd.points_from_xy( Metro_Art_in_Public_Places.Longitude,  Metro_Art_in_Public_Places.Latitude))

In [ ]:
type( Metro_Art_in_Public_Places_geo)

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip = gpd.sjoin(Metro_Art_in_Public_Places_geo, zipcodes, predicate = 'within')

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip.head()

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip.columns

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip = Metro_Art_in_Public_Places_geo_by_zip[['zip', 'Title', 'Artist Last Name', 'Artist First Name',
                                                                               'Location','Medium', 'Type', 'Description', 'Latitude', 
                                                                               'Longitude', 'City', 'State','x', 'y', 'geometry', 'index_right',
                                                                               'po_name']]

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip.head()

In [ ]:
Metro_Art_in_Public_Places_geo_by_zip['zip'].value_counts()

 ###  Let's take a closer look at the 33 Metro_Art_in_Public_Places in the 37219  zipcode

In [ ]:
Metro_Art_in_Public_Places_in_37219 = Metro_Art_in_Public_Places_geo_by_zip.loc[Metro_Art_in_Public_Places_geo_by_zip['zip'] == '37219']
Metro_Art_in_Public_Places_in_37219.shape

In [ ]:
polygon37219 = zipcodes.loc[zipcodes['zip'] == '37219']
polygon37219.shape

In [ ]:
#fig, ax = matplotlib.subplots()
ax = polygon37219.plot(figsize = (8, 10), color = 'lightgreen')
Metro_Art_in_Public_Places_in_37219.plot( ax = ax, column = 'Location');
plt.show();

In [ ]:
polygon37219.geometry.centroid

In [ ]:
#find the center of the larger (more southern) polygon and look at it
center = polygon37219.geometry.centroid[24]
print(center)

In [ ]:
# reverse the order when constructing the array for folium location
area_center = [center.y, center.x]

# check the order of area_center, the location for our folium map
print(area_center)

In [ ]:
#create our map of Nashville Zip Code 37219 and show it
map_37219 = folium.Map(location =  area_center, zoom_start = 12)
map_37219

In [ ]:
i = 0
for row_index, row_values in Metro_Art_in_Public_Places_in_37219.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
#draw our zip code area: 37219
map_37219 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37219).add_to(map_37219)

#iterate through stops_in_37219 to create locations and markers 
#for each loaction
#remember for Folium locations, lat is listed first!!

for row_index, row_values in Metro_Art_in_Public_Places_in_37219.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Location'])
    icon=folium.Icon(color="blue",icon="paint-brush", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_37219)
map_37219.save('../maps/map37219.html')

#display our map
map_37219

In [ ]:
map_37219 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37219).add_to(map_37219)
for row in Metro_Art_in_Public_Places_in_37219.iterrows():
    row_values = row[1] 
    location = [row_values['Latitude'], row_values['Longitude']]
    popup = (str(row_values['Artist Last Name']) + ': ' + 
             str(row_values['Type']) + '<br/>' +
             str(row_values['Medium'])).replace("'", "`")
    marker = folium.Marker(location = location, popup = popup)
    
    marker.add_to(map_37219)

#display our map
map_37219

In [ ]:
cluster_map_37219 = folium.Map(location =  area_center, zoom_start = 12)

#create a marker cluster
marker_cluster = MarkerCluster().add_to(cluster_map_37219)

folium.GeoJson(polygon37219).add_to(cluster_map_37219)

# inside the loop add each marker to the cluster
for row_index, row_values in Metro_Art_in_Public_Places_in_37219.iterrows():
    loc = [row_values['Latitude'], row_values['Longitude']]
    pop = str(row_values['Location'])
    icon=folium.Icon(color="blue",icon="paint-brush", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)

    
#save an interactive HTML map by calling .save()
cluster_map_37219.save('../maps/cluster37219.html')

cluster_map_37219

In [ ]:
#draw our zip code area: 37219
map_37219 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37219).add_to(map_37219)

#create a list of locations and pass them to FastMarkerCluster()
locations = Metro_Art_in_Public_Places_in_37219[['Latitude', 'Longitude']].values.tolist()
map_37219.add_child(
    FastMarkerCluster(locations)
)



#display our map
map_37219

In [ ]:
Metro_Art_in_Public_Places_in_37219.head(2)

In [ ]:
 Metro_Art_in_Public_Places_in_37219.columns

In [ ]:
Metro_Art_in_Public_Places_in_37219 = Metro_Art_in_Public_Places_in_37219.drop(columns=['zip', 'Description', 'Latitude', 'Longitude',
                                                  'State', 'x', 'y', 'geometry', 'index_right', 'po_name'])
Metro_Art_in_Public_Places_in_37219.head(2)

import pandas as pd

df = Metro_Art_in_Public_Places_in_37219

m = folium.Map([43, -100], zoom_start=4)

html = df.to_html(
    classes="table table-striped table-hover table-condensed table-responsive"
)

popup = folium.Popup(html)

folium.Marker([30, -100], popup=popup).add_to(m)

m

In [ ]:
import pandas as pd

df = Metro_Art_in_Public_Places_in_37219

map_37219 = folium.Map([36.16682 , -86.78317 ], zoom_start=12)

html = df.to_html(
    classes="table table-striped table-hover table-condensed table-responsive"
)

popup = folium.Popup(html)

folium.Marker([36.16682, -86.78317], popup=popup).add_to(map_37219)

map_37219

In [ ]:
import folium

# Create a map centered at specific coordinates
map_37219 = folium.Map(location=[36.16682, -86.78317], zoom_start=12)

# Replace this with your actual image URL
image_url = "https://static.wixstatic.com/media/cc4e3f_c26673969be949cabe37c3f3ed9e838e~mv2_d_5184_3456_s_4_2.jpg/v1/crop/x_864,y_0,w_3456,h_3456/fill/w_368,h_368,al_c,q_80,usm_0.66_1.00_0.01,enc_auto/IMG_9155_JPG.jpg"  # Ensure this is a valid image URL

# Create HTML for the popup to display the image
html = f'<img src="{image_url}" style="width:300px; height:auto;">'

# Add a marker with the image in the popup
folium.Marker([36.16682, -86.78317], popup=html, lazy=True).add_to(map_37219)

# Save the map to an HTML file
map_37219.save('map_with_marker.html')
map_37219

In [ ]:
import folium

# Create a map centered at specific coordinates
map_37219 = folium.Map(location=[36.16682, -86.78317], zoom_start=12)

# List of image URLs and their corresponding coordinates
images = [
    {
        "url": "https://static.wixstatic.com/media/cc4e3f_c26673969be949cabe37c3f3ed9e838e~mv2_d_5184_3456_s_4_2.jpg/v1/crop/x_864,y_0,w_3456,h_3456/fill/w_368,h_368,al_c,q_80,usm_0.66_1.00_0.01,enc_auto/IMG_9155_JPG.jpg",
        "location": [36.16682, -86.78317],
        "title": "Image 1"
    },
    {
        "url": "https://static.wixstatic.com/media/cc4e3f_9a63868c969e47479e9562691d28e374~mv2.png/v1/crop/x_0,y_20,w_1080,h_1040/fill/w_374,h_360,al_c,q_85,usm_0.66_1.00_0.01,enc_auto/Mural%20Guide%20for%20Web%20.png",
        "location": [36.17000, -86.78000],
        "title": "Image 2"
    },
    {
        "url": "https://i0.wp.com/walkslowrunwild.com/wp-content/uploads/2018/03/Stockholm-Stadion-Metro-Art-Rainbow.jpg?quality=80&strip=all&ssl=1",
        "location": [36.17100, -86.78500],
        "title": "Image 3"
    },
    {
        "url": "https://i0.wp.com/nashvillepublicart.com/wp-content/uploads/2021/11/Bottle-Mural-1.jpg?resize=1024%2C534&ssl=1",
        "location": [36.17200, -86.78600],
        "title": "Image 4"
    },
    {
        "url": "https://tse1.mm.bing.net/th?id=OIP.URzkFWnSqOROEUokTrSiOQHaEU&pid=Api&P=0&h=180",
        "location": [36.17300, -86.78700],
        "title": "Image 5"
    },
    {
        "url": "https://i0.wp.com/www.renetimmermans.com/blog/wp-content/uploads/2015/04/20150403-Stockholm-Sweden-0198-hdr.jpg?ssl=1",
        "location": [36.17400, -86.78800],
        "title": "Image 6"
    },
    {
        "url": "https://tse1.mm.bing.net/th?id=OIP.ctW6ul9lALc95JTwHjzO7wHaE8&pid=Api&P=0&h=180",
        "location": [36.17500, -86.78900],
        "title": "Image 7"
    },
    {
        "url": "https://tse2.mm.bing.net/th?id=OIP.TIyu3tGXv9brq3sA432tSgHaE8&pid=Api&P=0&h=180",
        "location": [36.17600, -86.79000],
        "title": "Image 8"
    },
    {
        "url": "https://tse1.mm.bing.net/th?id=OIP.zxiY3SegaI5-cFB8hb_UGwHaEU&pid=Api&P=0&h=180",
        "location": [36.17700, -86.79100],
        "title": "Image 9"
    },
    {
        "url": "https://tse3.mm.bing.net/th?id=OIP.MtnuGzO82RvNXHydTe_KLQHaEK&pid=Api&P=0&h=180",
        "location": [36.17800, -86.79200],
        "title": "Image 10"
    },
]

# Add markers for each image
for img in images:
    html = f'<img src="{img["url"]}" style="width:300px; height:auto;">'
    folium.Marker(img["location"], popup=html, tooltip=img["title"]).add_to(map_37219)

# Save the map to an HTML file
map_37219.save('map_with_multiple_images.html')
map_37219


In [ ]:
import folium

# Create a map centered at specific coordinates
map_37219 = folium.Map(location=[36.16682, -86.78317], zoom_start=12)

# List containing one image and one YouTube video with their coordinates
media = [
    {
        "type": "youtube",
        "url": "https://youtube.com/embed/cgvqfpkB3ZM",  # Replace with your actual YouTube video URL
        "location": [36.17000, -86.78000],
        "title": "YouTube Video 1"
    },
]

# Add markers for each media item
for item in media:
    if item["type"] == "image":
        html = f'<img src="{item["url"]}" style="width:300px; height:auto;">'
    elif item["type"] == "youtube":
        html = f'<iframe width="300" height="200" src="{item["url"]}" frameborder="0" allowfullscreen></iframe>'
    
    folium.Marker(item["location"], popup=html, tooltip=item["title"]).add_to(map_37219)

# Save the map to an HTML file
map_37219.save('map_with_image_and_youtube.html')
map_37219

In [ ]:
for ind, row in Metro_Art_in_Public_Places_in_37219.iterrows():
    if row['Type']in(['	Sculpture']): 
       Metro_Art_in_Public_Places_in_37219.loc[ind, 'Art_group'] = 'Sculpture'
    elif row['Type']in(['Frieze']) :
        Metro_Art_in_Public_Places_in_37219.loc[ind, 'Art_group'] = 'Frieze'
    elif row['Type']in(['Mural']):
        Metro_Art_in_Public_Places_in_37219.loc[ind, 'Art_group'] ='Mural'
    elif row['Type']in(['Mobile']):
      Metro_Art_in_Public_Places_in_37219.loc[ind, 'Art_group'] ='Mobile'

In [ ]:
Metro_Art_in_Public_Places_in_37219.dropna()